In [124]:
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

from symforce.notebook_util import display
import symforce.symbolic as sf
# 3.1.3 Show that the cross product is a lie algebra for a 3D vector
X = sf.Vector3.symbolic('X')
Y = sf.Vector3.symbolic('Y')
Z = sf.Vector3.symbolic('Z')

# Closure: 
X.cross(Y)



⎡X₁⋅Y₂ - X₂⋅Y₁ ⎤
⎢              ⎥
⎢-X₀⋅Y₂ + X₂⋅Y₀⎥
⎢              ⎥
⎣X₀⋅Y₁ - X₁⋅Y₀ ⎦

In [125]:
# Binlinear composition: 
a = sf.Symbol('a')
b = sf.Symbol('b')
V1 = (a*X + b*Y).cross(Z)
V2 = a*(X.cross(Z)) + b*(Y.cross(Z))
(V1-V2).simplify().transpose()


[0  0  0]

In [126]:
# Reflexive
X.cross(X).transpose()

[0  0  0]

In [127]:
# Jacobi identity: 
V1 = X.cross(Y.cross(Z)) + Z.cross(X.cross(Y)) + Y.cross(Z.cross(X))
V1.simplify().transpose()

[0  0  0]

In [128]:
# 3.1.4 Verify the lie bracket for so(3) meets the properties
def skew_to_vec(M: sf.M33)->sf.Vector3:
    return sf.Vector3(M[2, 1], M[0, 2], M[1, 0])

def LB(phia, phib):
    M = phia*phib.transpose() - phib*phia.transpose()
    return skew_to_vec(M)

phi_1 = sf.Vector3.symbolic("phi_1")
phi_2 = sf.Vector3.symbolic("phi_2")
phi_3 = sf.Vector3.symbolic("phi_3")

In [129]:
# Closure: 
type(LB(phi_1, phi_2))  # (should also be a Vector3)

symforce.geo.matrix.Matrix31

In [130]:
# Bilinear composition:
V1 = LB(a*phi_1 + b*phi_2, phi_3)
V2 = a*LB(phi_1, phi_3) + b*LB(phi_2, phi_3)
(V1-V2).simplify().transpose()

[0  0  0]

In [131]:
# Reflexive
LB(phi_1, phi_1).transpose()

[0  0  0]

In [132]:
# Jacobi
V = LB(phi_1, LB(phi_2, phi_3)) + LB(phi_3, LB(phi_1, phi_2)) + LB(phi_2, LB(phi_3, phi_1))
V.simplify().transpose()

[0  0  0]

In [140]:
# 3.1.5 Verify se(3) lie bracket meets the properties of lie algreba
rho_1 = sf.Vector3.symbolic('rho_1')
rho_2 = sf.Vector3.symbolic('rho_2')
rho_3 = sf.Vector3.symbolic('rho_3')
xi_1 = sf.Vector6.block_matrix([[rho_1], [phi_1]])
xi_2 = sf.Vector6.block_matrix([[rho_2], [phi_2]])
xi_3 = sf.Vector6.block_matrix([[rho_3], [phi_3]])

def make_skew(xi: sf.Vector6)->sf.M44:
    phi = sf.Vector3(xi[3:])
    rho = sf.Vector3(xi[:3])
    phi_hat = phi.skew_symmetric(phi)
    return sf.M44.block_matrix([
        [phi_hat, rho],
        [sf.Vector3.zeros(3,1).transpose(), sf.Vector1(0)] 
    ])

def skew_to_vec6(M: sf.M44)->sf.Vector6:
    rot_part = M[:3,:3]
    vec = skew_to_vec(rot_part)
    rho = M[:3, 3]
    return sf.Vector6.block_matrix([[rho], [vec]])

def se3LB(xi1: sf.V6, xi2: sf.V6)->sf.V6:
    temp = make_skew(xi1)*make_skew(xi2) - make_skew(xi2)*make_skew(xi1)
    return skew_to_vec6(temp)

In [141]:
# Closure - should be V6
type(se3LB(xi_1, xi_2))

symforce.geo.matrix.Matrix61

In [146]:
# Bilinear composition:
V1 = se3LB(a*xi_1 + b*xi_2, xi_3)
V2 = a*se3LB(xi_1, xi_3) + b*se3LB(xi_2, xi_3)
(V1-V2).simplify().transpose()

[0  0  0  0  0  0]

In [148]:
# Reflexive
se3LB(xi_1, xi_1).transpose()

[0  0  0  0  0  0]

In [153]:
# Jacobi identity: 
V = se3LB(xi_1, se3LB(xi_2, xi_3)) +\
    se3LB(xi_3, se3LB(xi_1, xi_2)) +\
    se3LB(xi_2, se3LB(xi_3, xi_1))
V.simplify().transpose()

[0  0  0  0  0  0]